In [33]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras
#!pip install imblearn

    100% |████████████████████████████████| 92kB 660kB/s 
  Running setup.py bdist_wheel for imbalanced-learn ... - \ | / done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/b8/20/bd/0b775f7e5d413ac72562b1a5126598bcb6e0eae10da659be9f
Successfully built imbalanced-learn


In [1]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from sklearn import metrics

Using TensorFlow backend.


## 1. DATA PREPARATION

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_flow.csv")
data.shape

(2339529, 35)

In [3]:
# add observation index
data["row_index"] = data.index

In [4]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

train: (2279790, 36)
test: (39821, 36)
known: (2319611, 36)
unknown: (19918, 36)


In [5]:
# List numeric features used as predictors
print(data.columns)
numVars = ["user_ratio_flow", "user_ratio_full", "listen_type", "first_flow",
           "song_plays", "artist_plays", "platform_name1", "platform_name2",
           "song_skips", "artist_skips", "song_session_position", "time_diff"] 

# Create the data input matrix that can be passed to the keras model
tr_data = tr[[column for column in tr.columns if column in numVars]].as_matrix()
ts_data = ts[[column for column in ts.columns if column in numVars]].as_matrix()
kn_data = kn[[column for column in kn.columns if column in numVars]].as_matrix()
un_data = un[[column for column in un.columns if column in numVars]].as_matrix()

Index(['user_id', 'media_id', 'artist_id', 'genre_id', 'context_type',
       'media_duration', 'listen_type', 'user_gender', 'user_age',
       'is_listened', 'sample_id', 'dataset', 'session_id',
       'song_session_position', 'first_flow', 'time_diff', 'hours',
       'is_listened_lag', 'genre_plays', 'genre_skips', 'artist_plays',
       'artist_skips', 'album_plays', 'album_skips', 'song_plays',
       'song_skips', 'user_ratio_flow', 'user_ratio_full', 'genre_ratio',
       'artist_ratio', 'song_ratio', 'platform_name1', 'platform_name2',
       'platform_family1', 'platform_family2', 'row_index'],
      dtype='object')


## 2. MODELING

### 2.1. INITIALIZING

In [6]:
# Create an input layer with embeddings
user_in    = Input(shape = (1,), dtype = 'int64',   name = "user_in")
song_in    = Input(shape = (1,), dtype = 'int64',   name = "song_in")
artist_in  = Input(shape = (1,), dtype = 'int64',   name = "artist_in")
context_in = Input(shape = (1,), dtype = 'int64',   name = "context_in")

# Create an input layer with numeric features
data_in = Input(shape = (tr_data.shape[1],), name = "data_in")

# Counting number of unique ID values
n_users   = tr.user_id.nunique()
n_songs   = tr.media_id.nunique()
n_artists = tr.artist_id.nunique()
n_context = tr.context_type.nunique()

# Create an embedding assigning k latent factors to each ID
u = Embedding(n_users,   50, input_length = 1, embeddings_regularizer = l2(1e-5))(user_in)
s = Embedding(n_songs,   50, input_length = 1, embeddings_regularizer = l2(1e-5))(song_in)
a = Embedding(n_artists, 50, input_length = 1, embeddings_regularizer = l2(1e-5))(artist_in)
c = Embedding(n_context, 50, input_length = 1, embeddings_regularizer = l2(1e-5))(context_in)

# Layer with embeddings
embedding_input = concatenate([u, c])
embedding_input = Flatten()(embedding_input)
embedding_dense = Dense(64, activation = "relu")(embedding_input)
embedding_dense = BatchNormalization()(embedding_dense)

# Layer with numeric features
data_dense = Dense(16, activation = "relu")(data_in)
data_dense = BatchNormalization()(data_dense)

# Constructing the further layers
x = concatenate([embedding_dense, data_dense])
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(32, activation='relu')(x) 
x = Dropout(0.5)(x)
output = Dense(1, activation = "sigmoid")(x)

# Specify the model that we want to use
model = Model([user_in, context_in, data_in], output)
model.compile(optimizer = "Adagrad", loss = "binary_crossentropy", metrics = ['accuracy'])

### 2.2. FIRST STAGE

In [7]:
# run the estimations on training data
model.fit([tr.user_id, tr.context_type, tr_data], tr.is_listened, 
validation_data = ([ts.user_id, ts.context_type, ts_data], ts.is_listened),
batch_size = int(len(tr)/100), epochs = 10)

Train on 2279790 samples, validate on 39821 samples
Epoch 1/10
2279790/2279790 [==============================] - 71s - loss: 0.5326 - acc: 0.7359 - val_loss: 0.6562 - val_acc: 0.6076
Epoch 2/10
2279790/2279790 [==============================] - 71s - loss: 0.4927 - acc: 0.7640 - val_loss: 0.6442 - val_acc: 0.6097
Epoch 3/10
2279790/2279790 [==============================] - 83s - loss: 0.4872 - acc: 0.7668 - val_loss: 0.6266 - val_acc: 0.6436
Epoch 4/10
2279790/2279790 [==============================] - 83s - loss: 0.4839 - acc: 0.7683 - val_loss: 0.6099 - val_acc: 0.6705
Epoch 5/10
2279790/2279790 [==============================] - 80s - loss: 0.4819 - acc: 0.7695 - val_loss: 0.6056 - val_acc: 0.6769
Epoch 6/10
2279790/2279790 [==============================] - 78s - loss: 0.4801 - acc: 0.7704 - val_loss: 0.6087 - val_acc: 0.6770
Epoch 7/10
2279790/2279790 [==============================] - 81s - loss: 0.4786 - acc: 0.7709 - val_loss: 0.6217 - val_acc: 0.6718
Epoch 8/10
2279790/22797

In [8]:
# predict validation data
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict([ts.user_id, ts.context_type, ts_data])
pred.to_csv(path + "pred_valid/keras_newdata_flow_hl.csv", index = False)
pred.head(5)

row_index  is_listened
2167       2167     0.991269
2168       2168     0.993327
2240       2240     0.993223
4314       4314     0.991834
4321       4321     0.992529

In [9]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred.is_listened)

0.72155204375222126

### 2.3. SECOND STAGE

In [10]:
# run the estimations on full known data
model.fit([kn.user_id, kn.context_type, kn_data], kn.is_listened,
batch_size = int(kn.shape[0]/100), epochs = 10)

Epoch 1/10
2319611/2319611 [==============================] - 84s - loss: 0.4779 - acc: 0.7709    
Epoch 2/10
2319611/2319611 [==============================] - 80s - loss: 0.4773 - acc: 0.7710    
Epoch 3/10
2319611/2319611 [==============================] - 80s - loss: 0.4767 - acc: 0.7716    
Epoch 4/10
2319611/2319611 [==============================] - 79s - loss: 0.4765 - acc: 0.7715    
Epoch 5/10
2319611/2319611 [==============================] - 80s - loss: 0.4760 - acc: 0.7716    
Epoch 6/10
2319611/2319611 [==============================] - 78s - loss: 0.4758 - acc: 0.7719    
Epoch 7/10
2319611/2319611 [==============================] - 78s - loss: 0.4753 - acc: 0.7719    
Epoch 8/10
2319611/2319611 [==============================] - 77s - loss: 0.4747 - acc: 0.7722    
Epoch 9/10
2319611/2319611 [==============================] - 80s - loss: 0.4746 - acc: 0.7725    
Epoch 10/10
2319611/2319611 [==============================] - 78s - loss: 0.4743 - acc: 0.7727    


In [11]:
# predict unknown data
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict([un.user_id, un.context_type, un_data])
pred = pred.sort_values("sample_id")
pred.to_csv(path + "pred_unknown/keras_newdata_flow_hl.csv", index = False)
pred.head(5)

sample_id  is_listened
2332218          0     0.981236
2182792          1     0.477413
2092769          2     0.660399
1806368          3     0.396212
2018166          4     0.841994